This interactive notebook will guide you through IgE analysis step by step and will reproduce graphs d and e in Fig. 4. Let us import all the necessary. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/SAbakumov/Decirculator.git

In [3]:
import numpy as np 
import pandas as pd 
import os 

try:
    data_folder = 'Decirculator/spt/data_spt/POST'
    from Decirculator.spt.tracking_utils import track_particles, find_multilobes, get_deltas_list
except:
    from spt.tracking_utils import track_particles, find_multilobes, get_deltas_list
    data_folder = 'spt/data_spt/POST'



First, let us load all the data. The raw locations data for POST DNP-BSA addition is located in data_spt/POST. In total, 6 movies were processed. We load the data and append the color of each particle into the last column

In [2]:
data_folder = 'Decirculator/spt/data_spt/POST'


raw_locations = {}
for processed_movie in os.listdir(data_folder):
    raw_locations[processed_movie] = []

    for color_enum, color in enumerate(['green','red','blue']):
        xy_locs = pd.read_csv(os.path.join(data_folder,processed_movie ,str(color)+'_predicted.csv')).to_numpy()
        xy_locs = np.hstack([xy_locs, color_enum*np.ones([xy_locs.shape[0],1])])
        raw_locations[processed_movie].append(xy_locs)



Now, let's track each of the datasets, so that we can produce tracks for each of the individual colors in each of the movies. The tracking requires a few parameters:
- Number of dark frames (mem_frame). -> This is the number of frames that a particle can 'disappear' within the track
- Conjugation radius (radius). -> This is the maximal distance that a particle can travel in between two localizations, even if there are dark frames inbetween.

The tracking is performed by global optimization of the total sum of distances between the candidate positions and previous tracks, which is calculated using a bipartite graph matching algorithm. The distance matrix and assignment takes into account the maximal possible travel distance.

In [3]:
max_distance = 4
mem_frame = 5

In [4]:
tracked_particles = {}
for processed_movie in os.listdir(data_folder):
    tracked_particles[processed_movie] = []
    for color_enum, color in enumerate(['green','red','blue']):
        tracked_particles[processed_movie]+=track_particles(raw_locations[processed_movie][color_enum],1, np.int64(np.max(raw_locations[processed_movie][color_enum][:,0])),mem_frame,max_distance)


100%|██████████| 995/995 [00:01<00:00, 672.33it/s]


Now that we have the tracks, we can see which particles are associated. To do so, we first need to filter out the non-sensical tracks. These are the tracks that contain <=2 total points. 

In [5]:
track_length = 2

for processed_movie in os.listdir(data_folder):
    tracked_particles[processed_movie] = [np.array(p) for p in tracked_particles[processed_movie] if len(p)>track_length]


It is time to find associated particles. Associated particles are defined as tracks that pass the colocalization criterium for >= 4 frames ( not subsequent frames). The co-localized time points are the ones that have <=1 px distance between them, and are both present in the frame.

In [6]:

double_lobes = {}
multilobes = {}

for processed_movie in os.listdir(data_folder):
    double_lobes[processed_movie], multilobes[processed_movie] = find_multilobes(tracked_particles[processed_movie])

C:\Users\Boris\AppData\Local\Programs\Python\Python39\lib\site-packages\numba\core\ir_utils.py:2139: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'particles' of function 'disambiguate_couples'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "spt\tracking_utils.py", line 73:
@njit
def disambiguate_couples(particles):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Now that we have separated multi-lobed particles from single-lobed particles, we can proceed and calculate all the statistics. We first should start by calculating the displacement list. These are all displacements with a certain delta t along each of the particles. This takes some time...

In [13]:
multilobed_particles = []
for key in multilobes.keys():
    multilobed_particles+=multilobes[key]

deltas_list_multilobes = get_deltas_list(multilobed_particles)


double_lobes_particles = []
for key in double_lobes.keys():
    double_lobes_particles+=double_lobes[key]

deltas_list_double_lobes = get_deltas_list(double_lobes_particles)
